# Curvy-CUSIPS: Swaps Relative Value

In [1]:
import sys
sys.path.append("../")

In [2]:
from CurvyCUSIPs.CurveDataFetcher import CurveDataFetcher
from CurvyCUSIPs.S490Swaps import S490Swaps
from CurvyCUSIPs.USTs import USTs
from CurvyCUSIPs.utils.pca_utils import calc_pca_loadings_matrix

from datetime import datetime
import QuantLib as ql
import pandas as pd
from sklearn.decomposition import PCA
from pandas.tseries.offsets import CustomBusinessDay 
from pandas.tseries.holiday import USFederalHolidayCalendar
from typing import Dict, List

import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
plt.style.use('ggplot')
params = {'legend.fontsize': 'x-large',
        'figure.figsize': (18, 10),
        'axes.labelsize': 'x-large',
        'axes.titlesize':'x-large',
        'xtick.labelsize':'x-large',
        'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import os
from dotenv import dotenv_values
env_path = os.path.join(os.getcwd(), "../.env")
config = dotenv_values(env_path)

import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

%load_ext autoreload
%autoreload 2

# Set up data fetching infra

In [3]:
curve_data_fetcher = CurveDataFetcher(fred_api_key=config["FRED_API_KEY"])

In [4]:
sofr_ois = S490Swaps(s490_curve_db_path=r"..\db\nyclose_sofr_ois")
usts = USTs(
    cusip_set_db_path=r"..\db\ust_cusip_set",
    cusip_timeseries_db_path=r"..\db\ust_cusip_timeseries",
    ct_eod_db_path=r"..\db\ust_eod_ct_yields",
    curve_data_fetcher=curve_data_fetcher,
)

..\db\nyclose_sofr_ois is behind --- cd into 'scripts' and run 'update_sofr_ois_db.py' to update --- most recent date in db: 2024-12-24 00:00:00
..\db\ust_cusip_set is behind --- cd into 'scripts' and run 'update_ust_cusips_db.py' to update --- most recent date in db: 2024-12-24 00:00:00


# Calc Fwd SOFR OIS Grids

In [180]:
start_date = datetime(2019, 1, 1)
end_date = datetime(2024, 12, 24)
bdates = pd.date_range(start=start_date, end=end_date, freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))

In [259]:
fwd_tenors = ["1M", "3M", "6M", "9M", "12M", "18M", "2Y", "3Y", "4Y", "5Y", "6Y", "7Y", "8Y", "9Y", "10Y", "15Y"]
fwd_grid_dict, ql_curves = sofr_ois.s490_nyclose_fwd_curve_matrices(
    start_date=start_date,
    end_date=end_date,
    ql_piecewise_method="logLinearDiscount",
    fwd_tenors=fwd_tenors,
)

Building Implied Fwd Curves...: 100%|██████████| 1498/1498 [00:15<00:00, 93.72it/s] 


In [275]:
dt = datetime(2024, 12, 24)
sofr_ois.fwd_grid_dict_curve_plotter(
    tenor_date_pairs=[("Spot", dt), ("3M Fwd", dt), ("12M Fwd", dt), ("5Y Fwd", dt)],
    fwd_grid_dict=fwd_grid_dict,
    use_plotly=True
)

In [274]:
tenors_to_plot = ["4Y Fwd 6Y-4Y Fwd 25Y"]
sofr_ois.fwd_grid_dict_timeseries_plotter(fwd_grid_dict=fwd_grid_dict, tenors_to_plot=tenors_to_plot, bdates=bdates, use_plotly=True)

PLOTTING SWAPS: 100%|██████████| 1498/1498 [00:00<00:00, 1547.48it/s]


In [272]:
pca_results = sofr_ois.pca_on_fwd_grids(
    start_date=datetime(2024, 6, 1),
    end_date=datetime(2024, 12, 24),
    fwd_grid_dict=fwd_grid_dict, 
    rm_swap_tenors=["1D", "1W", "2W", "3W", "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M"],
    indy_fwd_strips=True
)

df: pd.DataFrame = pca_results["rich_cheap_zscore_heatmap"]
df.style.background_gradient(cmap="RdYlGn", axis=0)

PCA ON INDY FWDs: 100%|██████████| 17/17 [00:08<00:00,  1.94it/s]


,Spot,1M Fwd,3M Fwd,6M Fwd,9M Fwd,12M Fwd,18M Fwd,2Y Fwd,3Y Fwd,4Y Fwd,5Y Fwd,6Y Fwd,7Y Fwd,8Y Fwd,9Y Fwd,10Y Fwd,15Y Fwd
Tenor,,,,,,,,,,,,,,,,,
12M,2.149388,0.763726,1.065749,1.565468,2.155453,2.495513,1.556129,-2.996409,-0.045627,-3.598267,-0.245251,-1.216184,-2.447144,-1.691501,-0.212906,-0.388212,0.325506
18M,-1.178454,0.663703,0.520780,-0.100265,-0.794592,-2.225486,-1.741869,-1.807812,-3.717152,-1.170607,-0.558627,-2.053729,0.622872,0.982600,0.137184,-0.439831,0.917396
2Y,-2.189550,-0.801493,-1.235200,-2.202749,-2.306345,-2.407157,-0.741943,1.927859,-3.470030,1.028620,-0.121083,-0.651536,1.817978,1.544301,0.293251,-0.465054,1.203710
3Y,-2.641326,-2.416018,-2.198831,-1.889035,-1.421360,-0.935583,-2.146257,0.060709,0.789478,1.463831,-1.129121,1.946755,2.748894,1.946067,0.288306,-0.386426,1.449287
4Y,-2.749335,-1.839386,-2.181174,-2.711561,-3.217222,-3.313869,-2.042985,2.147519,1.296662,0.975398,1.020845,2.840835,2.855317,1.437989,-0.192761,-0.139134,1.552991
5Y,-0.164206,-3.134208,-3.065034,-1.857038,-0.552637,0.613212,0.456576,2.240103,0.835558,2.896407,1.872837,3.290424,1.765772,0.704835,-0.609562,-0.083821,1.608730
6Y,1.539426,0.959623,1.127950,1.511568,1.240819,1.129773,0.479316,2.016668,2.606020,3.567496,2.046515,2.093483,0.896301,-0.525968,-0.695332,0.344358,-0.971745
7Y,2.056017,1.361512,1.415367,1.508716,1.253056,1.158019,1.417173,2.933212,3.252666,3.461354,1.520729,1.451513,-0.826589,-1.163895,-0.247501,1.013865,-1.078066
8Y,2.545392,1.526410,1.792013,2.215650,2.285567,2.306306,2.091289,3.440131,3.338450,2.136259,0.597549,-0.053278,-1.759759,-1.212703,0.441726,1.702696,-1.063559


In [266]:
most_mispriced_dict = sofr_ois.most_mispriced_pca_resid_zscores(df, top_n=50)
display(most_mispriced_dict["curve"])
display(most_mispriced_dict["fly"])

,Forward,Tenor1,Tenor2,ZScore-Spread,Trade
0,4Y Fwd,25Y,50Y,-7.795067,flattener
1,4Y Fwd,6Y,25Y,7.530997,steepener
2,4Y Fwd,12M,50Y,-7.429833,flattener
3,4Y Fwd,7Y,25Y,7.424855,steepener
4,4Y Fwd,12M,6Y,-7.165763,flattener
5,4Y Fwd,20Y,50Y,-7.105677,flattener
6,4Y Fwd,30Y,50Y,-7.063656,flattener
7,4Y Fwd,12M,7Y,-7.059621,flattener
8,3Y Fwd,18M,8Y,-7.055602,flattener
9,3Y Fwd,18M,7Y,-6.969818,flattener


,Forward,ShortWing,Belly,LongWing,ZScore-Spread,Trade
0,4Y Fwd,6Y,25Y,50Y,-15.326064,pay belly
1,4Y Fwd,7Y,25Y,50Y,-15.219922,pay belly
2,4Y Fwd,12M,6Y,25Y,14.696760,pay belly
3,4Y Fwd,5Y,25Y,50Y,-14.654975,pay belly
4,4Y Fwd,12M,7Y,25Y,14.484476,pay belly
5,4Y Fwd,12M,6Y,20Y,14.007370,pay belly
6,4Y Fwd,12M,6Y,30Y,13.965349,pay belly
7,4Y Fwd,6Y,20Y,50Y,-13.947284,pay belly
8,4Y Fwd,8Y,25Y,50Y,-13.894827,pay belly
9,4Y Fwd,6Y,30Y,50Y,-13.863241,pay belly
